In [ ]:
import json
import requests
import polars as pl
import logging
from typing import List, Dict

class TinyAPI:
    def __init__(self, token: str, logger_name: str = 'tiny_api'):
        self.token = token
        self.base_url = 'https://api.tiny.com.br/api2/'
        self.logger = logging.getLogger(logger_name)
        self.logger.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

        # File handler to save log messages
        file_handler = logging.FileHandler('tiny_api.log')
        file_handler.setFormatter(formatter)
        self.logger.addHandler(file_handler)

        # Stream handler to print log messages to console
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(formatter)
        self.logger.addHandler(stream_handler)

    def read_excel_to_dataframe(self, file_path: str) -> pl.DataFrame:
        try:
            df = pl.read_excel(file_path)
            self.logger.info("Excel sheet successfully read into DataFrame.")
            return df
        except Exception as e:
            self.logger.error(f"Error reading Excel sheet: {e}")
            raise

    def ensure_required_fields_present(self, df: pl.DataFrame) -> pl.DataFrame:
        if 'sku' not in df.columns and 'id' not in df.columns:
            self.logger.error("Neither 'sku' nor 'id' columns present in DataFrame.")
            raise ValueError("Either 'sku' or 'id' column must be present.")

        required_cols = ['sku', 'id']
        other_cols = [col for col in df.columns if col not in required_cols]

        if len(other_cols) == 0:
            self.logger.error("No additional columns present in DataFrame.")
            raise ValueError("At least one additional column must be present.")

        return df

    def search_product_info(self, identifier: str) -> Dict[str, str]:
        endpoint = 'produtos.pesquisa.php'
        url = f"{self.base_url}{endpoint}"
        params = {'token': self.token, 'formato': 'JSON', 'pesquisa': identifier}

        try:
            response = requests.post(url, data=params)
            if response.status_code == 200:
                product_info = json.loads(response.text)['retorno']
                if 'produtos' in product_info:
                    return product_info['produtos'][0]['produto']
                else:
                    self.logger.error(f"Product not found for identifier: {identifier}")
            else:
                self.logger.error(f"Error retrieving product info: {response.text}")
        except Exception as e:
            self.logger.error(f"Error: {e}")
        
        return None
        
    def get_product_info(self, id: str) -> Dict[str, str]:
        endpoint = 'produto.obter.php'
        url = f"{self.base_url}{endpoint}"
        params = {'token': self.token, 'formato': 'JSON', 'id': id}        

        try:
            response = requests.post(url, data=params) 
            if response.status_code == 200:
                product_info = json.loads(response.text)['retorno']                
                if 'produto' in product_info:
                    return product_info['produto']
                else:
                    self.logger.error(f"Product not found for id: {id}")
            else:
                self.logger.error(f"Error retrieving product info: {response.text}")
        except Exception as e:
            self.logger.error(f"Error: {e}")

        return None
    
    def update_products_info(self, df: pl.DataFrame) -> List[Dict[str, str]]:
        results = []

        for index, row in enumerate(df.rows()):
            df_product = dict(zip(df.columns, row))
            sku_index = df.get_column_index('sku')
            id_index = df.get_column_index('id')
            df_product_id = row[id_index] if id_index is not None else None

            if df_product_id is None:
                sku = row[sku_index]
                api_product_info = self.search_product_info(sku)
                product_id = api_product_info['id'] if api_product_info else None
            else:
                product_id = df_product_id
            
            required_product_info = self.get_product_info(product_id)

            if required_product_info:
                df_product['sequencia'] = str(index + 1)
                for key, value in required_product_info.items():
                    if key not in df_product:
                        df_product[key] = value
            else:
                self.logger.error(f"Product not found for id: {product_id}")
            
            results.append({'produto': df_product})

        return results

    def _update_products(self, products: List[Dict[str, str]]) -> List[Dict[str, str]]:
        endpoint = 'produto.alterar.php'
        url = f"{self.base_url}{endpoint}"
        products_json = json.dumps({'produtos': products})
        params = {'token': self.token, 'produto': products_json, 'formato': 'JSON'}        

        try:
            response = requests.post(url, data=params)
            if response.status_code == 200:
                response_text = json.loads(response.text)['retorno']                
                if 'status' in response_text and response_text['status'] == 'OK':
                    return response_text
                else:
                    self.logger.error(f"Error updating products: {response_text}")
            else:
                self.logger.error(f"Error updating products: {response.text}")
        except Exception as e:
            self.logger.error(f"Error: {e}")

    def update_products(self, products: List[Dict[str, str]], chunk_size: int = 20) -> List[Dict[str, str]]:
        results = []
        for i in range(0, len(products), chunk_size):
            chunk = products[i:i + chunk_size]
            result = self._update_products(chunk)
            results.append(result)
        return results

    def process_excel_sheet(self, file_path: str) -> List[Dict[str, str]]:
        try:
            df = self.read_excel_to_dataframe(file_path)
            df = self.ensure_required_fields_present(df)
            results = self.update_products(df)

            return results
        except Exception as e:
            self.logger.error(f"Error processing Excel sheet: {e}")
            return []

In [ ]:
token = '2dfe6aa48bf77f657d34167a2dcdd0a6f4a35da8'
file_path = 'ecosmeticos.xlsx'
tiny_api = TinyAPI(token)
df = tiny_api.read_excel_to_dataframe(file_path)
df = tiny_api.ensure_required_fields_present(df)
products_to_update = tiny_api.update_products_info(df)
# convert preco_min into 'obs' field on each product
for product in products_to_update:
    product['produto']['obs'] = f"preco_minimo: {product['produto']['preco_min']}"
    del product['produto']['preco_min']
result = tiny_api.update_products(products_to_update, chunk_size=20)

In [ ]:
print(json.dumps(result[0]['registros'], indent=4))

In [5]:
price = 'R$29,90'
#print currence symbol
print(price[:2])
#print integer part of price by spliting last 3 characters and first 2 characters
print(price[2:-3])
# print fractional part of price by spliting last 3 characters
print(price[-2:])


R$
29
90
